# 어텐션(Attention)

### 맥락 벡터 : 단어 벡터에 가중치를 를 곱하여 합한 가중합을 구한 벡터
단어를 선택하는 작업은 미분 불가하므로 모든 것을 선택하고 단어의 중요도를 가중치로 계산

In [1]:
import numpy as np
np.random.seed(100)

T,H = 5,4   # T : 시계열의 길이, H : Hidden size
hs = np.random.randn(T,H)  # (5,4)
print('hs:\n',hs)
print('hs0:',hs[0])

a  = np.array([0.8, 0.1, 0.03, 0.05, 0.02]) # 가중치
a.sum()  # 1.0
print('a:\n',a)

# (1) repeat() 함수 사용
ar = a.reshape(T,1).repeat(4,axis=1)
print('ar:\n',ar, ar.shape)

t = hs * ar
print('t:\n',t,t.shape)

# (2) Numpy 브로드캐스팅 사용, 1번과 결과 동일 
ar = a.reshape(T,1)  # (5,1)로 2차원으로 shape을 바꿈
print('ar:\n',ar, ar.shape)

t = hs * ar           # (5,4) * (5,1) : 브로드캐스팅 적용, , 단어벡터에 가중치를 곱함
print('t:\n',t,t.shape)

c = np.sum(t, axis=0)  # 수직 방향으로 합 , 가중합, 맥락 벡터
print('c:   ',c)
print('hs0: ',hs[0])
print(c.shape)

hs:
 [[-1.74976547  0.3426804   1.1530358  -0.25243604]
 [ 0.98132079  0.51421884  0.22117967 -1.07004333]
 [-0.18949583  0.25500144 -0.45802699  0.43516349]
 [-0.58359505  0.81684707  0.67272081 -0.10441114]
 [-0.53128038  1.02973269 -0.43813562 -1.11831825]]
hs0: [-1.74976547  0.3426804   1.1530358  -0.25243604]
a:
 [0.8  0.1  0.03 0.05 0.02]
ar:
 [[0.8  0.8  0.8  0.8 ]
 [0.1  0.1  0.1  0.1 ]
 [0.03 0.03 0.03 0.03]
 [0.05 0.05 0.05 0.05]
 [0.02 0.02 0.02 0.02]] (5, 4)
t:
 [[-1.39981238  0.27414432  0.92242864 -0.20194883]
 [ 0.09813208  0.05142188  0.02211797 -0.10700433]
 [-0.00568487  0.00765004 -0.01374081  0.0130549 ]
 [-0.02917975  0.04084235  0.03363604 -0.00522056]
 [-0.01062561  0.02059465 -0.00876271 -0.02236636]] (5, 4)
ar:
 [[0.8 ]
 [0.1 ]
 [0.03]
 [0.05]
 [0.02]] (5, 1)
t:
 [[-1.39981238  0.27414432  0.92242864 -0.20194883]
 [ 0.09813208  0.05142188  0.02211797 -0.10700433]
 [-0.00568487  0.00765004 -0.01374081  0.0130549 ]
 [-0.02917975  0.04084235  0.03363604 -0.0052205

In [2]:
# 3차원 배열의 가중합의 구현, 맥락 벡터
np.random.seed(100)

N, T, H = 10, 5, 4      # 면,행,열
hs = np.random.randn(N, T, H)
# print('hs:\n',hs)

np.random.seed(100)
a  = np.random.randn(N, T)  # 가중치의 합이 1이 아닌 예임
print('a:\n',a)

ar = a.reshape(N, T, 1).repeat(H, axis=2)  # 2번축(열)로 4번 반복
print('ar:\n',ar)

t = hs * ar  # (10,5, 4)
# print('t:\n',t)
print(t.shape)

c = np.sum(t, axis=1)  # 1번 축(행)으로 합 
print('c:\n',c)
print(c.shape)

a:
 [[-1.74976547  0.3426804   1.1530358  -0.25243604  0.98132079]
 [ 0.51421884  0.22117967 -1.07004333 -0.18949583  0.25500144]
 [-0.45802699  0.43516349 -0.58359505  0.81684707  0.67272081]
 [-0.10441114 -0.53128038  1.02973269 -0.43813562 -1.11831825]
 [ 1.61898166  1.54160517 -0.25187914 -0.84243574  0.18451869]
 [ 0.9370822   0.73100034  1.36155613 -0.32623806  0.05567601]
 [ 0.22239961 -1.443217   -0.75635231  0.81645401  0.75044476]
 [-0.45594693  1.18962227 -1.69061683 -1.35639905 -1.23243451]
 [-0.54443916 -0.66817174  0.00731456 -0.61293874  1.29974807]
 [-1.73309562 -0.9833101   0.35750775 -1.6135785   1.47071387]]
ar:
 [[[-1.74976547 -1.74976547 -1.74976547 -1.74976547]
  [ 0.3426804   0.3426804   0.3426804   0.3426804 ]
  [ 1.1530358   1.1530358   1.1530358   1.1530358 ]
  [-0.25243604 -0.25243604 -0.25243604 -0.25243604]
  [ 0.98132079  0.98132079  0.98132079  0.98132079]]

 [[ 0.51421884  0.51421884  0.51421884  0.51421884]
  [ 0.22117967  0.22117967  0.22117967  0.2211

### 가중합 WeightSum 계층 구현 : 맥락벡터를 구하는 계층

In [4]:
# 가중합을 구하는 class
class WeightSum:
    def __init__(self):
        self.params, self.grads = [],[]
        self.cache = None
        
    def forward(self, hs, a):  # hs : (N,T,H)
        N,T,H = hs.shape
        
        ar = a.reshape(N,T,1) # .repeat(H, axis=2)는 생략가능
        t = hs * ar
        c = np.sum(t, axis=1)  # (N,H)
        
        self.cache = (hs, ar)
        return c
    
    def backward(self,dc) :
        hs, ar = self.cache
        N,T,H = hs.shape
        
        dt = dc.reshape(N,1,H).repeat(T,axis=1)   # sum의 역전파, 출력: (N,T,H)
        dar = dt * hs       # (N,T,H)
        dhs = dt * ar       # (N,T,H)
        da = np.sum(dar, axis=2)    # repeat의 역전파, 출력 :(N,T)
        
        return dhs, da

### 가중치를 구하는 AttentionWeight 계층 구현: 각 단어의 가중치를 구하여 WeightSum 계층으로 전달한다

In [5]:
# nn_layers.py에 아래 Softmax class추가
# class Softmax:
#     def __init__(self):
#         self.params, self.grads = [], []
#         self.out = None

#     def forward(self, x):
#         self.out = softmax(x)
#         return self.out

#     def backward(self, dout):
#         dx = self.out * dout
#         sumdx = np.sum(dx, axis=1, keepdims=True)
#         dx -= self.out * sumdx
#         return dx

In [6]:
from nn_layers import softmax, Softmax

In [7]:
# 각단어의 가중치를 구하는 class
class AttentionWeight:
    def __init__(self):
        self.params, self.grads = [],[]
        self.softmax = Softmax()
        self.cache = None
        
    def forward(self, hs, h) :  # hs: (N,T,H),  h : (N,H)
        N,T,H = hs.shape
        
        hr = h.reshape(N,1,H) # .repeat(T,axis=1)는 생략가능
        t = hs * hr           #  (N,T,H)
        s = np.sum(t, axis=2) #  (N,T)
        a = self.softmax.forward(s)  # (N,T)
        
        self.cache = (hs,hr)
        return a
    
    def backward(self, da) :
        hs,hr = self.cache
        N,T,H = hs.shape
        
        ds = self.softmax.backward(da)    # (N,T)
        dt = ds.reshape(N,T,1).repeat(H,axis=2)  # (N,T,H)
        dhs = dt * hr              # (N,T,H)
        dhr = dt * hs              # (N,T,H)
        dh = np.sum(dhr, axis=1)   # (N,H)
        
        return dhs, dh     # (N,T,H) , (N,H)